<a href="https://colab.research.google.com/github/Chiwidude/ETL-EstimadorRiesgo/blob/dscience/ETLv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparación de Entorno**

In [ ]:
#ejecutar sino se tienen instaladas estas librerías
!pip install -U scikit-learn
!pip install scikit-learn-intelex
!pip install -U imbalanced-learn
!pip install xgboost
!pip install seaborn
!pip install Pyspatialml
!pip install scikit-learn-intelex
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.2 MB/s 
     |████████████████████████████████| 26.0 MB 54.8 MB/s 
     |████████████████████████████████| 300.3 MB 17 kB/s 
     |████████████████████████████████| 4.0 MB 39.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 5.5 MB/s 
     |████████████████████████████████| 199 kB 38.3 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

## Código de soporte

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Ploteo validación cruzada k-iteraciones
Esta rutina permite dibujar la comparación de resultados de la validación cruzada entre resultados de entrenamiento y resultados de validación para su análisis.

In [ ]:
def plot_cvalidation(x_label, y_label, train_data, val_data, ax):
        '''Rutina para dibujar un gráfico de barras agrupado mostrando datos de entrenamiento y validación
          resultantes del modelo en cada iteración aplicando validación cruzada de k -iteraciones.
         Parametros
         ----------
         x_label: str, 
            Nombre del algoritmo utilizado. ej:Random Forest
          
         y_label: str, 
            Nombre de la métrica a visualizar. ej: precisión
         plot_title: str, 
            Título del gráfico. ej: 'Precisión del modelo'
         
         train_result: list, array
            Resultados de la métrica en fase de entrenamiento del modelo en la validación cruzada.
        
         val_result: list, array
            Resultados de la métrica en fase de validación del modelo en la validación cruzada.
         Returns
         -------
         Gráfico de barras agrupado mostrando el resultado de la métrica deseada en cada iteración.
        '''        
        labels = ["1st Fold", "2nd Fold", "3rd Fold", "4th Fold", "5th Fold"]
        X_axis = np.arange(len(labels))
        ax.bar(X_axis-0.2, train_data, 0.4, color='blue', label='Training')
        for x,y in zip(X_axis-0.2, train_data):
            label = "{:.2f}".format(y)
            ax.annotate(label,
                       (x,y),
                 textcoords="offset points",
                 xytext=(0,2),
                 ha='center',
                       fontsize=10)
        ax.bar(X_axis+0.2, val_data, 0.4, color='red', label='Validation')
        for x,y in zip(X_axis+0.2, val_data):
            label = "{:.3f}".format(y)
            ax.annotate(label,
                       (x,y),
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center',
                       fontsize=10)
        ax.set_xticks(X_axis, labels)
        ax.set_xlabel(x_label, fontsize=10)
        ax.set_ylabel(y_label, fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True)

### Ploteo matríz de confusión
Rutina que permite dibujar la matriz de confusión de los modelos entrenados para analizar los resultados contra los resultados esperados.

In [ ]:
def plot_matrix_c(real_data, pred_data, title, color):
    '''
    Rutina para dibujar matriz de confusión de un modelo, mostrando resultado de las predicciones del modelo
    contra los valores reales de los datos.
    Parametros
    ----------
    real_data: array,
    conjunto de datos reales utilizados para la validación del modelo, con las etiquetas de los datos
    
    pred_data: array,
    conjunto de datos predichos por el modelo
    
    title: str,
    título del gráfico
    
    color: matplotlib color o object(cmap) ,
    Color a aplicar al gráfico, puede ser un str o objeto de los disponibles en matplotlib
    '''
    matrix = confusion_matrix(real_data, pred_data)
    matrix = matrix.astype('float') / matrix.sum(axis=1)[:,np.newaxis]

    plt.figure(figsize=(16,7))
    sns.set(font_scale=1.4)
    sns.heatmap(matrix, annot=True, annot_kws={'size':10},
                cmap=color, linewidths=0.2)

    class_names = ['no deslizamiento','deslizamiento']
    tick_marks = np.arange(len(class_names)) + 0.4
    tick_marks2 = tick_marks + 0.5
    plt.xticks(tick_marks, class_names, rotation=0)
    plt.yticks(tick_marks2, class_names, rotation=0)
    plt.xlabel('Clase predecida')
    plt.ylabel('Clase real')
    plt.title(title)
    plt.show()
    

### Ploteo gráfico - valor AUC-ROC

Rutina que dibuja el gráfico - valor AUC-ROC, comparando el resultado del modelo con parámetros predeterminados y el resultado con parámetros ajustados.

In [ ]:
def plot_auc_roc(real_data, pred_data_base, pred_data_adj):
    '''
    Rutina que dibuja los gráficos y valor AUC - ROC de los modelos con parámetros predeterminados
    y con parámetros ajustados.
    Parámetros
    ----------
    real_data: array, conjunto de datos reales para validación
    
    pred_data_base: array, conjunto de datos de las probabilidades de las predicciones hechas por el modelo con parámetros predeterminados
    
    pred_data_adj: array, conjunto de datos de las probabilidades de las predicciones hechas por el modelo  con parámetros ajustados
    '''
    fpr, tpr, _ = roc_curve(real_data, pred_data_base)

    auc = roc_auc_score(real_data, pred_data_base)

    #curva ROC parámetros pred.

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize= (16,6))

    axes[0].plot(fpr,tpr,label="AUC="+str(auc))
    axes[0].set_title('default params')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_xlabel('False Positive Rate')
    axes[0].legend(loc=4)

    fpr, tpr, _ = roc_curve(real_data, pred_data_adj)

    auc2 = roc_auc_score(real_data, pred_data_adj)

    #curva ROC parámetros ajustados

    axes[1].plot(fpr,tpr,label="AUC="+str(auc2))
    axes[1].set_title('params ajustados')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_xlabel('False Positive Rate')
    axes[1].legend(loc=4)
    plt.show()

### Búsqueda hiperparámetros
Esta rutina permite realizar la búsqueda de la mejor combinación de hiperparámetros dado un conjunto base, para el modelo indicado en los parámetros. Retornando la combinación encontrada por el módulo GridSearchCV

In [ ]:
def search_hyperparams(base_estimator, params_grid, train_data_x, train_data_y):
    '''
    Rutina que realiza la búsqueda de la mejor combinación de hiperparámetros para el modelo indicado.
    Retornando un diccionario con los hiperparámetros para la configuración del modelo.
    
    Parametros
    ----------
    estimator: modelo ML base  scikit-learn, instancia base del modelo para el cuál se hará la búsqueda de hiperparámetros.
    
    params_grid: dict, conjuntos de hiperpárametros para realizar la búsqueda de la mejor combinación
    
    train_data_x: array, conjunto de datos de entrenamiento para las variables independientes
    
    train_data_y: array, conjunto de datos de entrenamiento para la variable dependiente
    
    Retorna
    -------
    dict, Combinación de hiperpárametros encontrada por GridSearchCV
    '''
    search_model = GridSearchCV(estimator=base_estimator,
                                param_grid=params_grid,
                                cv=5, verbose=2, n_jobs = -1,
                          scoring=['f1','precision','recall'],
                          refit='recall')

    search_model.fit(train_data_x, train_data_y)
    
    return search_model.best_params_

### Entrenamiento Modelo
Esta rutina entrena la instancia de un modelo de ML scikit-learn, retornando el modelo ya entrenado.

In [ ]:
def train_model(model_configed, x_train_data, y_train_data):
    '''
    Rutina que entrena un modelo ML de scikit-learn ya configurado, con los datos indicados en los parámetros.
    
    Parámetros
    ----------
    model_configed: instancia modelo ML scikit-learn ya configurado con hiperparámetros sea ese el caso o una instancia
    predeterminada del modelo.
    
    x_train_data: array, datos de entrenamiento de las variables inpendientes
    
    y_train_data: array, datos de entrenamiento de las variables dependientes
    '''
    #Pipeline del modelo, realiza una normalización con StandarScaler y estos datos los pasa al modelo para entrenamiento
    model_ = Pipeline(
    [('scaling', StandardScaler()),
        ('classifier', model_configed)])
    model_.fit(x_train_data, y_train_data)
    return model_

## Ploteo mapa
Rutina que dibuja un mapa de la región con las predicciones realizadas por el modelo a través de pyspatialml.

In [ ]:
def plot_map(results):
    '''
    Rutina que dibuja mapa de las predicciones realizadas por el modelo.
    Parametros
    ----------
    results, rasterObject pyspatialML, resultados de la predicción realizada por el modelo
    '''
    fig, ax = plt.subplots(figsize=(9, 9))

    results.iloc[0].cmap = "Dark2"
    results.iloc[0].categorical = True
    results.iloc[0].plot(ax=ax, legend=True, categorical=True)
    plt.show()  

## Resultados validación cruzada
Ejecuta la evaluación de validación cruzada de un modelo dado, retornando un diccionario con los resultados de cada una de las iteraciones de la validación

In [ ]:
def cross_validation_results(estimator_, x_test_data, y_test_data, cv_):
    scores = cross_validate(estimator= estimator_,
    X = x_test_data,
    y = y_test_data,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    cv = cv_,
    return_train_score = True)
    return scores

# Ingeniería de Datos

In [ ]:
from sklearnex import patch_sklearn 
patch_sklearn()
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE
from pyspatialml import Raster
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
#Datos variables independendientes

files = ["drive/MyDrive/Tesis/raster_curvatura/curvatura_r.tif","drive/MyDrive/Tesis/raster_elevacion/elevacion_r.tif","drive/MyDrive/Tesis/raster_pendiente/pendiente_r.tif","drive/MyDrive/Tesis/raster_perfil_curvatura/perfilc_r.tif",
         "drive/MyDrive/Tesis/raster_lluvia/lluvia_r.tif","drive/MyDrive/Tesis/raster_cobertura/cobertura_r.tif"]

#Carga de datos con rasterio
rasters =Raster(files)

In [ ]:
#Datos deslizamientos
raster_desli = rasterio.open('drive/MyDrive/Tesis/deslizamientos/raster/desli_lim.tif')

In [ ]:
#Combinación de datos de deslizamientos y variables independientes (DATAFRAME)
extracted_df = rasters.extract_raster(raster_desli)
#Depuración de Datos
extracted_df.dropna(inplace=True)
extracted_df

,value,curvatura_r,elevacion_r,pendiente_r,perfilc_r,lluvia_r,cobertura_r,geometry
0,0.0,0.50,1621.0,9.800000,0.473218,2392.033447,2.0,POINT (369009.824 1777200.007)
1,0.0,0.75,1574.0,20.196274,0.687335,2648.619141,2.0,POINT (373376.451 1777200.007)
2,0.0,1.00,1640.0,2.977056,1.483140,2869.323730,2.0,POINT (378470.851 1777200.007)
3,0.0,0.75,594.0,21.701746,-3.096038,2679.779541,2.0,POINT (404670.618 1777200.007)
4,0.0,0.25,391.0,22.294275,0.006878,2923.886230,2.0,POINT (417770.501 1777200.007)
...,...,...,...,...,...,...,...,...
6159,0.0,1.25,894.0,11.518590,-0.665087,2843.120117,7.0,POINT (451247.982 1594529.407)
6160,0.0,0.50,820.0,1.662445,-0.142286,3109.098877,7.0,POINT (449792.439 1593801.636)
6162,0.0,0.25,767.0,6.109069,0.095966,3204.624512,2.0,POINT (449064.668 1593073.865)
6163,0.0,0.50,779.0,2.679327,0.239556,3208.293701,7.0,POINT (449792.439 1593073.865)


## Sobreajuste de Datos y separación de conjuntos

In [ ]:
#Separación de Datos inicial
#Sobremuestreo de datos
#separación variables independientes
X = extracted_df.drop(columns=['value','geometry']).values

#separación variable dependiente (deslizamientos)
Y = extracted_df['value'].values


#el módulo SMOTE, realiza el sobremuestreo del conjunto de datos haciendo uso de la técnica Synthetic Minority Oversampling Technique (SMOTE)
smote = SVMSMOTE(random_state=30, n_jobs=-1)

X_smote, Y_smote = smote.fit_resample(X,Y)

#Se hace la separación de datos de entrenamiento y datos de validación para las variables independientes y la variable dependiente
x_train, x_test, y_train, y_test = train_test_split(X_smote,Y_smote,test_size=0.2, random_state=30)

# Pipeline Ciencia de Datos (ML)

In [ ]:
from sklearnex import patch_sklearn 
patch_sklearn()
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import numpy as np

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Random Forest

In [ ]:
#Búsqueda mejor ajuste hiperparámetros
base_classifier = RandomForestClassifier()

n_estimators = [100,250,500,750,1000,1200]

max_depth = [50,100, 250, 350, 500]

min_samples_split = [5, 10, 15, 20, 30]

min_samples_leaf = [2,4,6,10]



params_grid = {'n_estimators':n_estimators,
               'max_depth':max_depth,
               'min_samples_split':min_samples_split,
               'min_samples_leaf':min_samples_leaf}

params = search_hyperparams(base_classifier, params_grid, x_train, y_train)

params['n_jobs'] = -1

params

### Entrenamiento Hiperparámetros ajustados


In [ ]:
#Instancia modelo con hiperparámetros
classifier = RandomForestClassifier(**params)


rf = train_model(classifier, x_train, y_train)

In [ ]:
#Resultados predicción (pyspatialml)
result_rf = rasters.predict(estimator=rf,file_path="drive/MyDrive/Tesis/prediction.tif", progress=True)
#Resultados probabilidades predicción (pyspatialml)
result_probs = rasters.predict_proba(estimator=rf, progress=True)

In [ ]:
#Plot mapa predicciones
plot_map(result_rf)

In [ ]:
#Estos resultados fueron utilizados para el análisis de métricas del modelo
#Resultados predicción (scikit-learn)
y_pred_ajustado = rf.predict(x_test)
#Resultados probabilidades predicción (scikit-learn)
y_ajustado_proba = rf.predict_proba(x_test)[::,1]

### Entrenamiento Hiperparámetros predeterminados


In [ ]:
#Instancia algoritmo hiperparámetros predeterminados
b_classifier = RandomForestClassifier(n_jobs=-1)

brf = train_model(b_classifier, x_train, y_train)

In [ ]:
#Resultados predicción (pyspatialml)
result_rfb = rasters.predict(estimator=brf, progress=True)
#Resultados probabilidades predicción (pyspatialml)
result_probs = rasters.predict_proba(estimator=brf, progress=True)

In [ ]:
#Plot mapa predicciones
plot_map(result_rfb)

In [ ]:
#Estos resultados fueron utilizados para análisis de métricas del modelo
#Resultados predicción (scikit-learn)
y_pred_base = brf.predict(x_test)
#Resultados probabilidades predicción (scikit-learn)
y_pbase_proba = brf.predict_proba(x_test)[::,1]

### Métricas

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

#### Matríz de Confusión + Métricas

In [ ]:
#Matriz de confusión parámetros ajustados
plot_matrix_c(y_test, y_pred_ajustado,'Matriz de Confusión Random Forest (hparam ajustados)',plt.cm.Blues)

In [ ]:
#Resultado métricas de clasificación modelo parámetros ajustados
print(classification_report(y_test, y_pred_ajustado, digits=5))

#### AUC - ROC

In [ ]:
plot_auc_roc(y_test, y_pbase_proba, y_ajustado_proba)

#### Validación Cruzada k - iteraciones

In [ ]:
#Validación modelo hiperparámetros pred.
base_scores = cross_validation_results(brf, x_test, y_test, 5)
for i, item in base_scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Validación modelo hiperparámetros ajustados
scores = cross_validation_results(rf, x_test, y_test, 5)
for i, item in scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Plot comparativa validación cruzada k - iteraciones RF
figure, axes = plt.subplots(nrows=4, ncols=2, figsize=(12,12))

metrics = ['accuracy', 'precision','recall','f1']

figure.suptitle('5-fold Validation', fontsize=24, fontweight='bold')

for index, metric in enumerate(metrics):
    pivot = 0
    plot_cvalidation('Random Forest (default params) test mean:'+str(round(base_scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',base_scores[f'train_{metric}'],base_scores[f'test_{metric}'], axes[index,pivot])
    pivot+=1
    plot_cvalidation(f'Random Forest (params ajustados)  test mean:'+str(round(scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',scores[f'train_{metric}'],scores[f'test_{metric}'], axes[index,pivot])
figure.tight_layout()
plt.show()

## Support Vector Machine

In [ ]:
#Búsqueda hiperparámetros
param_grid = {'C': [1,2,2.5,3,4,5],
              'gamma': [1,1.25,1.50,1.75],
              'kernel': ['rbf','linear','sigmoid'],
             'cache_size':[1000]}
base_svm = svm.SVC()

params_svm = search_hyperparams(base_svm, param_grid, x_train, y_train)

params_svm['probability'] = True

params_svm

In [ ]:
params_svm = {'C': 5,
              'gamma': 1.75,
              'kernel': 'rbf',
             'cache_size':1000,
              'probability': True
}

### Entrenamiento Hiperparámetros ajustados

In [ ]:
#Instacia de modelo con hiperparámetros
svmc = svm.SVC(**params_svm)

svc = train_model(svmc, x_train, y_train)

In [ ]:
#Resultado Predicción (pyspatialml)
result = rasters.predict(estimator=svc, progress=True)
#Resultado Probabilidades Predicción (pyspatialml)
result_probs = rasters.predict_proba(estimator=svc, progress=True)

In [ ]:
#Plot mapa con predicciones
plot_map(result)

In [ ]:
#Estos resultados fueron utilizados para el análisis de métricas del modelo
#Resultados predicción (scikit-learn)
y_pred_svm = svc.predict(x_test)
#Resultados probabilidades predicción (scikit-learn)
y_svm_proba = svc.predict_proba(x_test)[::,1]

### Entrenamiento Hiperparámetros predeterminados

In [ ]:
#Instancia modelo parámetros predeterminados
base_svm = svm.SVC(probability=True)
base_svm = train_model(base_svm, x_train, y_train)

In [ ]:
#Resultado Predicción (pyspatialml)
resultb = rasters.predict(estimator=base_svm, progress=True)
#Resultado Probabilidades Predicción (pyspatialml)
result_probs = rasters.predict_proba(estimator=base_svm, progress=True)

In [ ]:
#Plot mapa predicciones
plot_map(resultb)

In [ ]:
#Estos resultados fueron utilizados para el análisis de métricas del modelo
#Resultados predicción (scikit-learn)
y_pred_svmb = base_svm.predict(x_test)
#Resultados probabilidades predicción (scikit-learn)
y_svmb_proba = base_svm.predict_proba(x_test)[::,1]

### Métricas

#### Matríz de Confusión + métricas

In [ ]:
#Resultado métricas de validación modelo hparams. ajustados
print(classification_report(y_test, y_pred_svmb,digits=5))

In [ ]:
#Matriz confusión modelo hparams. ajustados
plot_matrix_c(y_test, y_pred_svm,'Matriz de Confusión SVM (hparam ajustados)',plt.cm.Blues)

#### AUC - ROC

In [ ]:
#Gráfico comparativa AUC - ROC
plot_auc_roc(y_test, y_svmb_proba, y_svm_proba)

#### Validación cruzada k-iteraciones

In [ ]:
#Validación modelo hparámetros pred.
base_scores = cross_validation_results(base_svm, x_test, y_test, 5)

for i, item in base_scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Validación modelo hparámetros ajustados
scores = cross_validation_results(svc, x_test, y_test, 5)

for i, item in scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Comparación validación cruzada k-iteraciones
figure, axes = plt.subplots(nrows=4, ncols=2, figsize=(12,12))

metrics = ['accuracy', 'precision','recall','f1']

figure.suptitle('5-fold Validation', fontsize=24, fontweight='bold')

for index, metric in enumerate(metrics):
    pivot = 0
    plot_cvalidation('SVM (default params) test mean:'+str(round(base_scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',base_scores[f'train_{metric}'],base_scores[f'test_{metric}'], axes[index,pivot])
    pivot+=1
    plot_cvalidation(f'SVM (params ajustados)  test mean:'+str(round(scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',scores[f'train_{metric}'],scores[f'test_{metric}'], axes[index,pivot])
figure.tight_layout()
plt.show()